install yfinance


In [18]:
%pip install yfinance
%pip install apscheduler
%pip install pandas
%pip install oandapyV20
%pip install oanda_candles

import the modules


In [6]:
import yfinance as yf
import pandas as pd

# Adjust the start and end dates
start_date = "2024-01-01"  # Example: Within the last 60 days
end_date = "2024-03-04"    # Example: Today's date

# Download data
dataF = yf.download("EURUSD=X", start=start_date, end=end_date, interval='1h')

# Print the data
#print(dataF)

dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-01-01 18:00:00+00:00,1.105583,1.105583,1.105583,1.105583,1.105583,0
2024-01-01 19:00:00+00:00,1.105583,1.105583,1.105339,1.105339,1.105339,0
2024-01-01 20:00:00+00:00,1.105339,1.105339,1.103875,1.104240,1.104240,0
2024-01-01 21:00:00+00:00,1.104240,1.105217,1.104240,1.105217,1.105217,0
2024-01-01 22:00:00+00:00,1.104850,1.105094,1.104606,1.104728,1.104728,0
...,...,...,...,...,...,...
2024-03-01 18:00:00+00:00,1.084011,1.084481,1.083424,1.084363,1.084363,0
2024-03-01 19:00:00+00:00,1.084246,1.084599,1.084128,1.084481,1.084481,0
2024-03-01 20:00:00+00:00,1.084363,1.084363,1.083776,1.084128,1.084128,0


2. Defining your signal functions

In [11]:
def signal_generator(df):
    #we will select the last (element) row of the dataframe--like we get the 
    #last (element) of a Open, Close from the database we made above and pass them to open, close.
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    #do the same here but we get the previous two last (element) row of the dataframe.
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]

    #Bearish pattern....selling signal
    if (open>close and
    previous_open<previous_close and
    close<previous_open and
    open>=previous_close) :
        return 1
    
    #Bullish pattern...buying sgnal
    if(open<close and
    previous_open>previous_close and
    close>previous_open and
    open<=previous_close):
        return 2

    #No clear pattern
    else:
        return 0

#create a list of signals based on the two patterns from the signal_generator function
#for each pair of consecutive rows in the Dataframe.
#create an empty list and append 0 to it as the function above requires two rows of data
#to compare and generate a signal. so we want the first row not to have a signal and to begin in the second row.
signal = []
signal.append(0)
#loop over the dataframe dataF from the second row to the last row
for i in range(1, len(dataF)):
    df = dataF[i-1:i+1]#this line creates a new dataframe that contains the current row and the previous row from dataF(last two rows of the dataframe)
    signal.append(signal_generator(df))#this line appends the signal generated from the signal_generator function to the signal list
#signal_generator(data)
dataF["signal"] = signal

In [13]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

signal
0    950
2     59
1     44
Name: count, dtype: int64

3. Connect to the market and execute the trades.

In [17]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

ModuleNotFoundError: No module named 'oanda_candles'